In [1]:
import pandas as pd
import numpy as np
from pydataset import data

In [2]:
mpg = data('mpg')

In [3]:
mpg.head()
mpg['avg_mpg'] = (mpg.cty + mpg.hwy) / 2

In [4]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,avg_mpg
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0


In [5]:
make_list = list(mpg.manufacturer.unique())

avg_make_mpg = pd.DataFrame(
    ([(make, mpg.avg_mpg[mpg.manufacturer == make].sum() / 
        len(mpg.avg_mpg[mpg.manufacturer == make])) 
        for make in make_list]),
    columns = ['make', 'avg_mpg']
    )
avg_make_mpg[avg_make_mpg.avg_mpg == avg_make_mpg.avg_mpg.max()]
     
     

,make,avg_mpg
4,honda,28.5


In [6]:
avg = pd.DataFrame(mpg.groupby('manufacturer').mean().avg_mpg)
avg.sort_values(by='avg_mpg', ascending=False).head(1)

,avg_mpg
manufacturer,
honda,28.5


In [7]:
len(mpg.manufacturer.value_counts())

15

In [8]:
len(mpg.model.unique())

38

In [9]:
print(f'Automatic cars have an average of \
{mpg.avg_mpg[mpg.trans.str.contains("auto")].mean():.2f} mpg')

Automatic cars have an average of 19.13 mpg


In [10]:
print(f'Manual cars have an average of \
{mpg.avg_mpg[mpg.trans.str.contains("manual")].mean():.2f} mpg')

Manual cars have an average of 22.23 mpg


In [11]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})

In [12]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})

In [13]:
pd.merge(users, roles, left_on='role_id', right_on='id', how='left')

,id_x,name_x,role_id,id_y,name_y
0,1,bob,1.0,1.0,admin
1,2,joe,2.0,2.0,author
2,3,sally,3.0,3.0,reviewer
3,4,adam,3.0,3.0,reviewer
4,5,jane,NaN,NaN,NaN
5,6,mike,NaN,NaN,NaN


In [14]:
# db_name = 'employees'
# url = f'mysql+pymysql://{user}:{password}@{host}/{db_name}'
# db = pd.read_sql('SELECT * FROM employees LIMIT 5', url)

In [15]:
def get_db_url(user, host, password, db_name):
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [25]:
from env import user, host, password

url = get_db_url(user, host, password, 'employees')
employees = pd.read_sql('SELECT * FROM employees', url)
titles = pd.read_sql('SELECT * FROM titles', url)

In [30]:
merged = pd.merge(employees, titles, left_on='emp_no', right_on='emp_no')

In [50]:
merged.title.unique()

array(['Senior Engineer', 'Staff', 'Engineer', 'Senior Staff',
       'Assistant Engineer', 'Technique Leader', 'Manager'], dtype=object)